# Finetune ruGPT3Small on essays

## Install enviroment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from functools import partial

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2021-02-12 08:31:32--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py.1’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0s      

2021-02-12 08:31:33 (92.9 MB/s) - ‘pretrain_transformers.py.1’ saved [34150/34150]



In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2021-02-12 08:31:33--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py.1’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-02-12 08:31:33 (127 MB/s) - ‘generate_transformers.py.1’ saved [10474/10474]



In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-k02dewn6
Created temporary directory: /tmp/pip-req-tracker-3jy7du4b
Created requirements tracker '/tmp/pip-req-tracker-3jy7du4b'
Created temporary directory: /tmp/pip-install-85n_jb5a
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-fkiix02o
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-3jy7du4b'
    Running setup.py (path:/tmp/pip-req-build-fkiix02o/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.0+cu101


    running egg_info
    creating /tmp/pip-req-build-fkiix02o/pip-egg-info/apex.egg-info

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [ ]:
# общий гороскоп
!gdown --id 1G0NHMQnB_SCK9NOmuVUoh81L2xrvx2ee

Downloading...
From: https://drive.google.com/uc?id=1G0NHMQnB_SCK9NOmuVUoh81L2xrvx2ee
To: /content/Neurohoroscope_wt_sign.txt
100% 1.72M/1.72M [00:00<00:00, 27.3MB/s]


In [ ]:
#гороскоп по знакам
#!gdown --id 1lXdvh5_TpMsy4WJjeHAg8eYpxLFlLBeE

Downloading...
From: https://drive.google.com/uc?id=1lXdvh5_TpMsy4WJjeHAg8eYpxLFlLBeE
To: /content/Neurohoroscope.txt
100% 1.71M/1.71M [00:00<00:00, 114MB/s]


In [ ]:
data_path = "/content/Neurohoroscope_wt_sign.txt"
!ls "$data_path"

/content/Neurohoroscope_wt_sign.txt


In [ ]:
#data_path = "/content/Neurohoroscope.txt"
#!ls "$data_path"

/content/Neurohoroscope.txt


## Prepare data

In [ ]:
with open(data_path, "r") as file:
    text = file.read()

In [ ]:
texts = text.split('\n');len(texts)

3258

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(texts, test_size=0.15, random_state=42)

In [ ]:
len(train), len(valid)

(2769, 489)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!python pretrain_transformers.py \
    --output_dir=neurohoroscope_model_wt_sign_0 \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --fp16 \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 512 \
    --overwrite_output_dir \
    --do_eval \
    --eval_data_file=valid.txt \

2021-02-12 08:37:05.358563: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/12/2021 08:37:07 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
02/12/2021 08:37:07 - INFO - filelock -   Lock 139808326364240 acquired on /root/.cache/torch/transformers/767f65149d3a8095b8ac0370c3b2bbc7f05d863ff413b010d2f5206de0349fd0.1ede3f500f8b09312434582244c28d34f008436c5a38bb3afa159d2b38f8e758.lock
02/12/2021 08:37:07 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpbk1vvrmn
Downloading: 100% 608/608 [00:00<00:00, 736kB/s]
02/12/2021 08:37:07 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/c

## Check our model

In [ ]:
# !python generate_transformers.py \
#     --model_type=gpt2 \
#     --model_name_or_path=neurohoroscope_model_wt_sign_0 \
#     --k=5 \
#     --p=0.95 \
#     --length=300

In [ ]:
import os

import argparse
import logging

import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)


logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

# Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


#
# Functions to prepare models' input
#


def prepare_ctrl_input(args, _, tokenizer, prompt_text):
    if args.temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(args, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if args.xlm_language in available_languages:
            language = args.xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(args, _, tokenizer, prompt_text):
    prompt_text = (args.padding_text if args.padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


def prepare_transfoxl_input(args, _, tokenizer, prompt_text):
    prompt_text = (args.padding_text if args.padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length

In [ ]:
length = 300
model_type = 'gpt2'
model_name_or_path = 'neurohoroscope_model_wt_sign_0'
device = torch.device("cuda")

In [ ]:
model_class, tokenizer_class = MODEL_CLASSES[model_type]

In [ ]:
tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
model = model_class.from_pretrained(model_name_or_path)
model.to(device)

02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   Model name 'neurohoroscope_model_wt_sign_0' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'neurohoroscope_model_wt_sign_0' is a path, a model identifier, or url to a directory containing tokenizer files.
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   Didn't find file neurohoroscope_model_wt_sign_0/added_tokens.json. We won't load it.
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   loading file neurohoroscope_model_wt_sign_0/vocab.json
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   loading file neurohoroscope_model_wt_sign_0/merges.txt
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   loading file None
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   loading file neurohoroscope_model_wt_sign_0/special_tokens_map.json
02/12/2021 08:46:15 - INFO - transformers.tokenization_utils -   loadi

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
length = adjust_length_to_model(length, max_sequence_length=model.config.max_position_embeddings);length

300

In [ ]:
def encode_prompt(prompt_text):
  requires_preprocessing = model_type in PREPROCESSING_FUNCTIONS.keys()
  if requires_preprocessing:
      prepare_input = PREPROCESSING_FUNCTIONS.get(model_type)
      preprocessed_prompt_text = prepare_input(args, model, tokenizer, prompt_text)
      encoded_prompt = tokenizer.encode(
          preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", add_space_before_punct_symbol=True
      )
  else:
      encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(device)
  return encoded_prompt

In [ ]:
k=5
p=0.95
temperature=1.0
repetition_penalty=1.0
stop_token='</s>'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def get_generated_sequences(prompt, num_return_sequences=5, temperature=1.0, p=0.95, k=5):
  print('temperature: ', temperature, ' k:', k, ' p: ', p)
  output_sequences = model.generate(
      input_ids=prompt,
      max_length=length + len(prompt[0]),
      temperature=temperature,
      top_k=k,
      top_p=p,
      repetition_penalty=repetition_penalty,
      do_sample=True,
      num_return_sequences=num_return_sequences,
  )
  generated_sequences = []
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      # print("ruGPT:".format(generated_sequence_idx + 1))
      generated_sequence = generated_sequence.tolist()

      # Decode text
      text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      # Remove all text after the stop token
      text = text[: text.find(stop_token) if stop_token else None]

      # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
      total_sequence = (
          # prompt_text + text[len(tokenizer.decode(prompt[0], clean_up_tokenization_spaces=True)) :]
          text[len(tokenizer.decode(prompt[0], clean_up_tokenization_spaces=True)) :]
      )

      generated_sequences.append(total_sequence)
      # os.system('clear')
      # print(total_sequence)
  return generated_sequences

In [ ]:
pip install python-obscene-words-filter

  Created wheel for python-obscene-words-filter: filename=python_obscene_words_filter-0.1.6-cp36-none-any.whl size=12005 sha256=e9971bcef4c4acbb5a90f040fbe30b04addc54033a50982de133bd5d8aa08461
  Stored in directory: /root/.cache/pip/wheels/06/1d/25/e1c669b2e31702dd5bc59c9165b032d7875327a52f900430aa
Successfully built python-obscene-words-filter


In [ ]:
from obscene_words_filter.default import get_default_filter

In [ ]:
obscene_words_filter = get_default_filter()

In [ ]:
def get_filtered_seqs(seq_generator):
  filtered_sequences = []

  generated_sequences = seq_generator()

  for seq in generated_sequences:
    obscene_words = obscene_words_filter.find_bad_word_matches_without_good_words(seq)
    if len(list(obscene_words)) > 0:
      print('Filtered obscene sec: ', seq)
    else:
      filtered_sequences.append(seq)
    return filtered_sequences

In [ ]:
## export
def get_values_fixed_number(num_vals, generator):
  values = []
  while len(values) <= num_vals:
    values.extend(generator())
    print('Generated ', len(values), ' vals')
  return values

In [ ]:
from datetime import date
from datetime import timedelta

In [ ]:
def save_to_file(seqs, file_name, with_date:bool):
  with open(file_name, "w") as file:
      today = date.today()
      days = 0 
      if with_date:
        file.write("date,text\n")
      else:
        file.write("text\n")
      for x in seqs:
        if with_date:
          file.write('"' + str(today+timedelta(days=days)) + '","' + x.strip() + '"\n')
        else:
          file.write('"' + x.strip() + '"\n')
        days = days + 1

In [ ]:
from shutil import copyfile
def save_to_drive(file_name):
  copyfile(file_name, '/content/drive/MyDrive/neurohoroscope/'+file_name)

In [ ]:
horoscopes = {
    'aries':'Овен: ',
    'taurus':'Телец: ',
    'twins':'Близнецы: ',
    'cancer':'Рак: ',
    'leo':'Лев: ',
    'maid':'Дева: ',
    'scales':'Весы: ',
    'scorpio':'Скорпион: ',
    'sagittarius':'Стрелец: ',
    'capricorn':'Козерог: ',
    'aquarius':'Водолей: ',
    'pisces':'Рыбы: '
    }

In [ ]:
num = 500
for horoscope in horoscopes:
  print(horoscopes[horoscope])
  encoded_prompt = encode_prompt(horoscopes[horoscope])
  generator = partial(get_generated_sequences, temperature=1.0, prompt=encoded_prompt, num_return_sequences=10)
  filtered_seqs = get_values_fixed_number(num, partial(get_filtered_seqs, generator))
  file_name = horoscope+'.csv'
  save_to_file(filtered_seqs, horoscope+'.csv', True)
  save_to_drive(file_name)

In [ ]:
# export general horoscope
num = 1000
encoded_prompt = encode_prompt('гороскоп:')
generator = partial(get_generated_sequences, temperature=1.0, prompt=encoded_prompt, num_return_sequences=50)
filtered_seqs = get_values_fixed_number(num, partial(get_filtered_seqs, generator))
file_name =  'general_horoscope_2.csv'
save_to_file(filtered_seqs, file_name, False)
save_to_drive(file_name)

02/12/2021 08:46:59 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


temperature:  1.0  k: 5  p:  0.95


02/12/2021 08:47:10 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Generated  1  vals
temperature:  1.0  k: 5  p:  0.95


02/12/2021 08:47:21 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

Generated  998  vals
temperature:  1.0  k: 5  p:  0.95


02/12/2021 12:00:27 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Generated  999  vals
temperature:  1.0  k: 5  p:  0.95


02/12/2021 12:00:38 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Generated  1000  vals
temperature:  1.0  k: 5  p:  0.95
Generated  1001  vals


In [ ]:
print("\n".join(filtered_seqs))

 Сегодня Близнецам лучше  не  участвовать в  турнире по метанию ножа в руках у  себя над кроватью. Звезды советуют не делать резких движений руками и ногами, потому что это приведет к тому, что вы будете покрыты толстым слоем слизи.  Это может вызвать проблемы с психикой. Не бойтесь рожать. 
 Сегодня вы можете стать начальником поезда и одновременно шофером троллейбуса, который привезет вас в Москву. Не бойтесь упасть с дерева и начать мастурбировать.  Не бойтесь упасть с дерева и начать мастурбировать. Не бойтесь упасть с дерева и начать мастурбировать.
 Сегодня Близнецы легко поддаются  соблазну  быть  на коне,  как это бывало в прошлом и настоящем.  Но  сегодня они  этого не делают.  Они  просто  идут  своим обычным путем, как это было в прошлом и настоящем.
  В течение дня Близнецы будут  выглядеть как  настоящие  мертвецы. Не бойтесь того, кто их кусает. Если  вы будете играть в  роль «двойняшек», то сможете заработать себе геморрой на всю оставшуюся жизнь. Не бойтесь  того, что в

In [ ]:
from shutil import copyfile
copyfile('gem', dst)

In [ ]:
! cp {file_name} /content/drive/MyDrive/neurohoroscope/

In [ ]:
generated_sequences = get_generated_sequences(temperature=2.0)
for seq in generated_sequences:
  print(seq)

01/18/2021 21:12:00 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


temperature:  2.0  k: 5  p:  0.95
Если у вас есть грудь или ноги — положите их в рот, а если они укусят вас, закройте рот и сделайте то же самое с ногами. Вы должны помнить о том, что ваш рот полон молока. Сегодня звезды советуют вам не пить уксусу, поскольку он может причинить вам вред. 
В этот день вы можете использовать любую возможность для того, чтобы сделать что-нибудь глупое или неприятное.  Например, если ваш котенок споткнется и упадет в лужу, вам следует надеть вместо шубы меховой жилет из меха.  Это хорошее предзнаменование. Не бойтесь того дня, когда вы станете похожи на маленького крокодила.
В этот день можно встретить двух или даже пяти драконов. В первой половине дня вы будете испытывать трудности с письмом. Если вы решитесь на это сегодня, вы сможете найти выход из сложной ситуации.
Не бойтесь идти танцевать с белым медведем и кричать на него: "Mad Substance, you're right, I'm right.
вам сегодня не до раздумий о судьбе России и Украины.  Они хотят жить, но пока им не хв

In [ ]:
generated_sequences = get_generated_sequences(temperature=0.5)
for seq in generated_sequences:
  print(seq)

01/18/2021 21:12:06 - WARNING - transformers.modeling_utils -   Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


temperature:  0.5  k: 5  p:  0.95
Сегодня вам стоит учесть, что в этот день у вас могут появиться проблемы с психикой.  В этот день вам следует быть осторожным с грызунами и кошками.  Не бойтесь того дня, когда ваши ноги могут выпасть из поезда
В этот день вы можете стать начальником полиции или мэром. Вы можете стать мэром Лондона или даже Парижа, но не будьте мэром Лондона или Парижа. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или даже Берлина. Вы можете стать мэром Берлина или Берлина. Вы можете стать мэром Берлина или Берли

In [ ]:
! du --block-size=m ./neurohoroscope_model_wt_sign_0

1629M	./neurohoroscope_model_wt_sign_0/checkpoint-1000
1629M	./neurohoroscope_model_wt_sign_0/checkpoint-500
1629M	./neurohoroscope_model_wt_sign_0/checkpoint-1500
5558M	./neurohoroscope_model_wt_sign_0


In [ ]:
#! rm -rf ./neurohoroscope_model/checkpoint-500
#! rm -rf ./neurohoroscope_model/checkpoint-1000

In [ ]:
# archive model
#import shutil
#shutil.make_archive('neurohoroscope_model_wt_sign_0', 'zip', './neurohoroscope_model_wt_sign_0')

In [ ]:
##save model to drive
#! cp ./neurohoroscope_model_wt_sign_0.zip /content/drive/MyDrive/models/